In [24]:
import os
import simulation
import pynbody
import pandas as pd
import matplotlib.pylab as plt
import matplotlib
import numpy as np
import ipywidgets
import tqdm
from simulation.sfh_in_box import sfh, plot_sfh, plot_binned_sfh, bin_sfh
from astropy import constants as c
from astropy.table import Table
from simulation.luminosity import surface_brightness, color_plot
from simulation.angmom import sideon
from simulation.derived import feh, mgfe, gas_metals
from simulation.parsers.parse_info import parse_info
from simulation.util import get_sim_name, make_df_monotonic_again_using_info, get_no_gti_intervals, prune_no_gti

In [2]:
from simulation.parsers.parse_trace import parse_dens_trace

In [3]:
sim_dir = "/media/michele/My Book/Michele/MySimulations/MovingBox/np/mb.41002_p300_a800_r600_t9.56/out"

In [4]:
sim = simulation.Simulation(sim_dir)

loading simulation: /media/michele/My Book/Michele/MySimulations/MovingBox/np/mb.41002_p300_a800_r600_t9.56/out
Found parameter file
Found compiler file
Found 445 snapshots
Loading cosmological parameters
Found cached trace file
trace.txt file is non-monotonic. Trying to recover
Found dens_temp_trace file
dens_temp_trace.txt file is non-monotonic. Trying to recover


In [5]:
info = parse_info(os.path.join(sim_dir, 'info.txt'))

In [7]:
len(info), len(sim.trace), len(sim._trace_orig)

(1081123, 1080274, 1081124)

In [8]:
# sim.trace.to_pickle('trace_41p3_mono_itself.pkl')

In [9]:
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1081123 entries, 0 to 1081122
Data columns (total 4 columns):
step    1081123 non-null int64
t       1081123 non-null float64
z       1081123 non-null float64
dt      1081123 non-null float64
dtypes: float64(3), int64(1)
memory usage: 33.0 MB


In [10]:
sim._trace_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1081124 entries, 0 to 1081123
Data columns (total 13 columns):
t           1081124 non-null float64
x           1081124 non-null float64
y           1081124 non-null float64
z           1081124 non-null float64
vx          1081124 non-null float64
vy          1081124 non-null float64
vz          1081124 non-null float64
ax_adhoc    1081124 non-null float64
ay_adhoc    1081124 non-null float64
az_adhoc    1081124 non-null float64
r           1081124 non-null float64
v           1081124 non-null float64
a_adhoc     1081124 non-null float64
dtypes: float64(13)
memory usage: 107.2 MB


In [11]:
info.head()

,step,t,z,dt
0,0,9.56205,0.408909,0.000000
1,1,9.56206,0.408908,0.000008
2,2,9.56207,0.408907,0.000008
3,3,9.56208,0.408906,0.000008
4,4,9.56209,0.408905,0.000008


In [12]:
sim._trace_orig.head()

,t,x,y,z,vx,vy,vz,ax_adhoc,ay_adhoc,az_adhoc,r,v,a_adhoc
0,9.56205,599.868,0.000000,0.000000e+00,-389.089,554.82,0.000000,0.000991,-0.000666,-0.002274,599.868,677.654397,0.002568
1,9.56206,599.864,0.004696,-7.152080e-11,-389.096,554.82,-0.000017,0.000990,-0.000668,-0.002273,599.864,677.658417,0.002568
2,9.56207,599.861,0.009393,-2.862650e-10,-389.103,554.82,-0.000034,0.000989,-0.000671,-0.002279,599.861,677.662436,0.002574
3,9.56208,599.858,0.014089,-6.444160e-10,-389.111,554.82,-0.000051,0.000989,-0.000671,-0.002279,599.858,677.667029,0.002574
4,9.56209,599.855,0.018786,-1.145980e-09,-389.118,554.82,-0.000068,0.000989,-0.000671,-0.002279,599.855,677.671049,0.002574


In [13]:
info.tail(20)

,step,t,z,dt
1081103,1080307,13.9999,0.0,0.000004
1081104,1080308,13.9999,0.0,0.000004
1081105,1080309,13.9999,0.0,0.000004
1081106,1080310,13.9999,0.0,0.000004
1081107,1080311,13.9999,0.0,0.000004
1081108,1080312,13.9999,0.0,0.000004
1081109,1080313,13.9999,0.0,0.000004
1081110,1080314,13.9999,0.0,0.000004
1081111,1080315,13.9999,0.0,0.000004
1081112,1080316,13.9999,0.0,0.000004


In [14]:
sim.trace.tail(20)

,t,x,y,z,vx,vy,vz,ax_adhoc,ay_adhoc,az_adhoc,r,v,a_adhoc
1081104,13.9999,284.643,110.344,0.524799,-188.389,1090.71,3.19767,0.000707,-0.001592,0.002100,305.283002,1106.864465,0.002728
1081105,13.9999,284.643,110.349,0.524813,-188.397,1090.70,3.19768,0.000707,-0.001592,0.002100,305.284809,1106.855973,0.002728
1081106,13.9999,284.642,110.353,0.524826,-188.404,1090.70,3.19768,0.000706,-0.001590,0.002100,305.285323,1106.857164,0.002727
1081107,13.9999,284.641,110.358,0.524840,-188.412,1090.70,3.19768,0.000706,-0.001590,0.002100,305.286198,1106.858526,0.002727
1081108,13.9999,284.640,110.362,0.524853,-188.420,1090.69,3.19769,0.000706,-0.001590,0.002100,305.286711,1106.850034,0.002727
1081109,13.9999,284.639,110.367,0.524867,-188.427,1090.69,3.19769,0.000706,-0.001590,0.002100,305.287587,1106.851226,0.002727
1081110,13.9999,284.639,110.372,0.524880,-188.435,1090.69,3.19769,0.000706,-0.001590,0.002100,305.289394,1106.852588,0.002727
1081111,13.9999,284.638,110.376,0.524894,-188.443,1090.69,3.19770,0.000706,-0.001590,0.002100,305.289908,1106.853950,0.002727
1081112,14.0000,284.637,110.381,0.524908,-188.450,1090.68,3.19770,0.000706,-0.001590,0.002100,305.290783,1106.845287,0.002727
1081113,14.0000,284.636,110.385,0.524921,-188.458,1090.68,3.19770,0.000706,-0.001590,0.002100,305.291297,1106.846649,0.002727


In [16]:
trace_mono_using_info = make_df_monotonic_again_using_info(sim._trace_orig[:-1], info)

In [17]:
trace_mono_using_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1080327 entries, 0 to 1081122
Data columns (total 13 columns):
t           1080327 non-null float64
x           1080327 non-null float64
y           1080327 non-null float64
z           1080327 non-null float64
vx          1080327 non-null float64
vy          1080327 non-null float64
vz          1080327 non-null float64
ax_adhoc    1080327 non-null float64
ay_adhoc    1080327 non-null float64
az_adhoc    1080327 non-null float64
r           1080327 non-null float64
v           1080327 non-null float64
a_adhoc     1080327 non-null float64
dtypes: float64(13)
memory usage: 115.4 MB


In [48]:
test_info = pd.DataFrame({'step': [0, 1, 2, 1, 2, 3, 4, 5, 3, 4, 5, 6, 7]})

In [49]:
test_info

,step
0,0
1,1
2,2
3,1
4,2
5,3
6,4
7,5
8,3
9,4


In [57]:
idx_restart, restart_points = get_no_gti_intervals(test_info)

In [58]:
idx_restart

[1, 5]

In [59]:
restart_points.index

Int64Index([3, 8], dtype='int64')

In [60]:
prune_no_gti(test_info, idx_restart, restart_points)

,step
0,0
3,1
4,2
8,3
9,4
10,5
11,6
12,7


In [21]:
trace_mono_using_info.t.is_monotonic_increasing

True

In [18]:
sim.trace.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1080274 entries, 0 to 1081123
Data columns (total 13 columns):
t           1080274 non-null float64
x           1080274 non-null float64
y           1080274 non-null float64
z           1080274 non-null float64
vx          1080274 non-null float64
vy          1080274 non-null float64
vz          1080274 non-null float64
ax_adhoc    1080274 non-null float64
ay_adhoc    1080274 non-null float64
az_adhoc    1080274 non-null float64
r           1080274 non-null float64
v           1080274 non-null float64
a_adhoc     1080274 non-null float64
dtypes: float64(13)
memory usage: 115.4 MB


In [9]:
info.step.is_monotonic_increasing

False

All together

In [ ]:
def make_info_monotonic_again(info):
    diff = info.step.diff(period)
    restart_points = info.step[diff < 0]
    idx_restart = [idx - ((info.step.loc[idx - 1]) - v + 1) for idx, v in zip(restart_points.index, restart_points.values)]

    na = info.copy()
    for a, b in zip(idx_restart, restart_points.index):
        na = na.drop(info.index[slice(a,b)])
    return na


In [11]:
diff = self.trace.t.diff()

In [12]:
self.trace.t[(diff < 0)]

241507     10.5317
423355     11.1021
605703     11.8780
791405     12.9376
953184     13.6765
1044241    13.8916
Name: t, dtype: float64